In [25]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install opensearch-py --quiet
!pip install tiktoken --quiet
!pip install psycopg2-binary --quiet
!pip install pgvector --quiet
!pip install unstructured --quiet

In [3]:
import os
os.environ['OPENAI_API_KEY']=open("openai-key").read()

In [23]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from langchain.document_loaders import TextLoader, UnstructuredFileLoader
from langchain.docstore.document import Document

In [39]:
loader = UnstructuredFileLoader("Israel.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

print("split_docs size:",len(docs))
print(docs[0])

embeddings = OpenAIEmbeddings()

Created a chunk of size 1436, which is longer than the specified 100
Created a chunk of size 1480, which is longer than the specified 100
Created a chunk of size 1461, which is longer than the specified 100
Created a chunk of size 1407, which is longer than the specified 100
Created a chunk of size 1421, which is longer than the specified 100
Created a chunk of size 1184, which is longer than the specified 100
Created a chunk of size 1143, which is longer than the specified 100
Created a chunk of size 1469, which is longer than the specified 100
Created a chunk of size 1084, which is longer than the specified 100


split_docs size: 10
page_content='In 2021, the government of Israel\xa0announced\xa0its selection of AWS as its primary cloud provider as part of the Nimbus contract for government ministries and subsidiaries. The Nimbus framework provides cloud services to Israeli government ministries—including local municipalities, government-owned companies, and public sector organizations—with the aim of driving innovation and enabling new digital services for the citizens of Israel. “We are excited to welcome the new AWS Region to Israel as part of project Nimbus and we congratulate AWS for its commitment and hard work in building the Israeli Region,” said Yali Rothenberg, accountant general of Israel. “The establishment of the Israeli Region reflects the long-term commitment of AWS to Israel and to the continuation of their large investments in the Israeli market. The establishment of the Region will enable us to migrate substantial governmental workloads to the cloud, and we are confident that 

In [28]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg2"),
    host=os.environ.get("PGVECTOR_HOST", "vector-store.co2wcr3kjcuz.us-east-1.rds.amazonaws.com"),
    port=int(os.environ.get("PGVECTOR_PORT", "5432")),
    database=os.environ.get("PGVECTOR_DATABASE", "postgres"),
    user=os.environ.get("PGVECTOR_USER", "sean"),
    password=os.environ.get("PGVECTOR_PASSWORD", "xxxxxxxx"),
)

In [29]:
COLLECTION_NAME = "Israel"

db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [34]:
question = "how many Availability Zones does AWS have?"
similarDocs = db.similarity_search(question, include_metadata=True,k=1)
[print(x) for x in similarDocs]

page_content='AWS Regions are composed of Availability Zones that place infrastructure in separate and distinct geographic locations. The AWS Israel (Tel Aviv) Region consists of three Availability Zones, each located far enough from the other to support customers’ business continuity, but near enough to provide low latency for high availability applications that use multiple Availability Zones. Each Availability Zone has independent power, cooling, and physical security and is connected through redundant, ultra-low latency networks. AWS customers focused on high availability can design their applications to run in multiple Availability Zones to achieve even greater fault tolerance. The launch of the AWS Israel (Tel Aviv) Region enables customers to securely store data in Israel while providing customers with even lower latency across the country to drive greater productivity, more efficient business operations, and enhanced real-time application performance. Customers also have access

[None]